In [36]:
# Basic initialization
%matplotlib inline
import math
import numpy as np
from scipy.linalg import cholesky
import matplotlib.gridspec as gridspec

import qiskit
from qiskit import Aer, execute
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile
import sys
import os

sys.path.append("/Users/bpnachman/Desktop/dl/lattice_qft")
print(sys.path)

from lattice_qft.core import lattice as lattice_cf
#from lattice_qft.core import settings
from lattice_qft.Scalar_Field_Theory import basic_operator_implementations as basic_op_cf
from lattice_qft.Scalar_Field_Theory import lattice_sft

import itertools

['/Users/bpnachman/Desktop/dl/lattice_qft/lattice_qft/examples', '/Users/bpnachman/opt/anaconda3/envs/Tutorial/lib/python311.zip', '/Users/bpnachman/opt/anaconda3/envs/Tutorial/lib/python3.11', '/Users/bpnachman/opt/anaconda3/envs/Tutorial/lib/python3.11/lib-dynload', '', '/Users/bpnachman/opt/anaconda3/envs/Tutorial/lib/python3.11/site-packages', '/Users/bpnachman/Desktop/dl/lattice_qft', '/Users/bpnachman/Desktop/dl/lattice_qft', '/Users/bpnachman/Desktop/dl/lattice_qft']


In [9]:
# Define the lattice to be used
nQ = 2
dim = 1
num_ancilla = 1
dx = 1
nL = 3
QC_lattice = lattice_sft.sft_lattice(dim, nL, dx, nQ, num_ancilla)

In [4]:
from qiskit import IBMQ

In [6]:
IBMQ.load_account() # Load account from disk
print(IBMQ.providers())    # List all available providers
provider = IBMQ.get_provider(group='open')

ibmqfactory.load_account:WARNING:2023-06-22 00:19:12,068: Credentials are already in use. The existing account in the session will be replaced.


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]


In [7]:
print(provider.backends())

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_jakarta') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_lagos') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibm_nairobi') from I

In [42]:
QC_nL = 3 # Number of lattice sites
QC_dx = 1 # Lattice spacing, usually called a in the lattice literature
QC_nQ = 2 # Number of qubits per lattice site
QC_dim = 1
QC_num_ancilla = 1

for myback in provider.backends():
    
    if myback.configuration().n_qubits < 6:
        continue
    
    QC_lattice = lattice_cf.Lattice(QC_dim, QC_nL, QC_dx, QC_nQ, QC_num_ancilla)
    cr = ClassicalRegister(QC_nL*QC_nQ)

    #Ground state
    qc_ground = QuantumCircuit(lattice.get_q_register())
    QC_state_prep = lattice.ground_state(qc_ground, lattice.get_q_register())
    QC_state_prep.measure_all()

    mapped_circuit = transpile(QC_state_prep, backend=backend)
    
    if 'cx' in mapped_circuit.count_ops():
        print(myback,mapped_circuit.count_ops()['cx'])

ibmq_qasm_simulator 137
simulator_statevector 137
simulator_mps 137
simulator_extended_stabilizer 137
simulator_stabilizer 137
ibmq_jakarta 137
ibm_lagos 137
ibm_nairobi 137
ibm_perth 137


In [39]:
backend = provider.get_backend('ibm_perth')

QC_nL = 3 # Number of lattice sites
QC_dx = 1 # Lattice spacing, usually called a in the lattice literature
QC_nQ = 2 # Number of qubits per lattice site
QC_dim = 1
QC_num_ancilla = 1

QC_lattice = lattice_cf.Lattice(QC_dim, QC_nL, QC_dx, QC_nQ, QC_num_ancilla)
cr = ClassicalRegister(QC_nL*QC_nQ)

#Ground state
qc_ground = QuantumCircuit(lattice.get_q_register())
QC_state_prep = lattice.ground_state(qc_ground, lattice.get_q_register())
QC_state_prep.measure_all()

mapped_circuit = transpile(QC_state_prep, backend=backend)

In [43]:
#mapped_circuit.draw()

In [ ]:
from qiskit.providers.ibmq.managed import IBMQJobManager
job_manager = IBMQJobManager()
job_set = job_manager.run(circs, backend=backend, name='myjob', shots=8192)
print(job_set.job_set_id())